<a href="https://colab.research.google.com/github/olcaykursun/ML/blob/main/neuralnets/transfer_learning_questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Answer the following questions based on the incorrect code below:

1. Fix Gross Errors in the Program:
   - Identify and correct the issues in the program, such as unsuitable augmentation layers, incorrect loss function, incorrect number of color channels for MobileNetV2, and the missing use of preprocess_input for the pretrained model.
   - The Sequential model is inherently meant to add layers one by one in order, either in a list or with the .add() method, whereas the Functional approach allows for more flexible graph-like connections, which doesn’t fit well with the Sequential style. Fix it.

2. Data Augmentation:
   - Are the data augmentation layers appropriate for the MNIST dataset? Would the RandomFlip('vertical') augmentation make sense for digit classification?
   - Given that MobileNetV2 was pretrained on ImageNet, does adding data augmentation layers help improve the performance of the model when applied to the MNIST dataset? Why or why not?

3. Training:
   - The base model's trainable attribute is set to True. Should the base model be trainable in this case, considering the goal of transfer learning? Why or why not?
   - The final layers added to the model include a Dense(64, activation='relu') and a Dropout(0.2) layer. Is this sufficient for classifying the MNIST digits? Would you recommend different layers or settings to improve model performance?

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2, preprocess_input
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical



# There is a three-line spacing between code segments below, and each segment contains at least one bug to fix.



# Load the MobileNetV2, which is a faster and more lightweight model, suitable for quicker training and inference.
base = MobileNetV2(weights='imagenet', include_top=True, input_shape=(32, 32, 1))



# Freeze the pretrained layers
base.trainable = True



# Build the new model using Functional API incorrectly mixed with Sequential
model = models.Sequential()
input_tensor = Input(shape=(32, 32, 3))
x = base(input_tensor)  # Using Functional approach
# Add the MobileNetV2 base
model.add(base(x))
# Adding data augmentation layers to Sequential model
model.add(layers.RandomFlip('horizontal'))
model.add(layers.RandomFlip('vertical'))
model.add(layers.RandomRotation(0.1))
# Flatten and add output layers
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.2))  # Regularization to prevent overfitting
model.add(layers.Dense(10, activation='relu'))



# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])



# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Preprocess the data
x_train = tf.image.resize(tf.expand_dims(x_train, axis=-1), [32, 32])
x_test = tf.image.resize(tf.expand_dims(x_test, axis=-1), [32, 32])
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)



# Train the model
model.fit(x_train, y_train, epochs=5, batch_size=32, validation_data=(x_test, y_test))
# Evaluate the model
test_loss, test_acc = model.evaluate(x_test)
print(f'Test accuracy: {test_acc:.4f}')